In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, glob
import string
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download()

stopwords = nltk.corpus.stopwords.words('english')
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

# Preprocess Data

In [2]:
file_loc ={'neg' : "./Data/train/neg/", 'pos' : "./Data/train/pos"}
text_neg = []
text_pos = []
for file in glob.glob(os.path.join(file_loc['neg'], "*.txt")):
    f = open(file, 'r', encoding = "utf8")
    text_neg.append(f.read())
    f.close()
for file in glob.glob(os.path.join(file_loc['pos'], "*.txt")):
    f = open(file, 'r', encoding = "utf8")
    text_pos.append(f.read())
    f.close()
    

KeyboardInterrupt: 

In [18]:
no = ["no"]*len(text_neg)
yes = ["yes"]*len(text_pos)
text = text_neg + text_pos
sent = no+yes
dict = {"review":text, "sentiment":sent}
df = pd.DataFrame()
df.to_csv("train.csv")

In [2]:
data = pd.read_csv('train.csv')

In [4]:
data['sentiment'].value_counts()

no     12500
yes    12500
Name: sentiment, dtype: int64

In [3]:
def remove_punctuation(text):
    return "".join([i for i in text if i not in string.punctuation])
def tokenise(text):
    return re.split('\W+', text)
def remove_stopwords(text):
    return [i for i in text if i not in stopwords]
def stemming(text):
    return [porter_stemmer.stem(word) for word in text]
def lemmatising(text):
    lemm = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm

In [4]:
data['punc_removed'] = data['review'].apply(lambda x: remove_punctuation(x))
data['lower'] = data['punc_removed'].apply(lambda x: x.lower())
data['tokenised'] = data['lower'].apply(lambda x: tokenise(x))
data['no_stopwords'] = data['tokenised'].apply(lambda x: remove_stopwords(x))
data['stemmed'] = data['no_stopwords'].apply(lambda x: stemming(x))
data['lemmatized'] = data['stemmed'].apply(lambda x: lemmatising(x))
data.head()

,Unnamed: 0,review,sentiment,punc_removed,lower,tokenised,no_stopwords,stemmed,lemmatized
0,0,Story of a man who has unnatural feelings for ...,no,Story of a man who has unnatural feelings for ...,story of a man who has unnatural feelings for ...,"[story, of, a, man, who, has, unnatural, feeli...","[story, man, unnatural, feelings, pig, starts,...","[stori, man, unnatur, feel, pig, start, open, ...","[stori, man, unnatur, feel, pig, start, open, ..."
1,1,Airport '77 starts as a brand new luxury 747 p...,no,Airport 77 starts as a brand new luxury 747 pl...,airport 77 starts as a brand new luxury 747 pl...,"[airport, 77, starts, as, a, brand, new, luxur...","[airport, 77, starts, brand, new, luxury, 747,...","[airport, 77, start, brand, new, luxuri, 747, ...","[airport, 77, start, brand, new, luxuri, 747, ..."
2,2,This film lacked something I couldn't put my f...,no,This film lacked something I couldnt put my fi...,this film lacked something i couldnt put my fi...,"[this, film, lacked, something, i, couldnt, pu...","[film, lacked, something, couldnt, put, finger...","[film, lack, someth, couldnt, put, finger, fir...","[film, lack, someth, couldnt, put, finger, fir..."
3,3,"Sorry everyone,,, I know this is supposed to b...",no,Sorry everyone I know this is supposed to be a...,sorry everyone i know this is supposed to be a...,"[sorry, everyone, i, know, this, is, supposed,...","[sorry, everyone, know, supposed, art, film, w...","[sorri, everyon, know, suppos, art, film, wow,...","[sorri, everyon, know, suppos, art, film, wow,..."
4,4,When I was little my parents took me along to ...,no,When I was little my parents took me along to ...,when i was little my parents took me along to ...,"[when, i, was, little, my, parents, took, me, ...","[little, parents, took, along, theater, see, i...","[littl, parent, took, along, theater, see, int...","[littl, parent, took, along, theater, see, int..."


In [5]:
dict = {'sentiment': data['sentiment'], 'review': data['stemmed']}
df = pd.DataFrame(dict)
df.to_csv("train_compiled.csv")

In [6]:
data = pd.read_csv("train_compiled.csv")

In [7]:
print(data['review'][0])

['stori', 'man', 'unnatur', 'feel', 'pig', 'start', 'open', 'scene', 'terrif', 'exampl', 'absurd', 'comedi', 'formal', 'orchestra', 'audienc', 'turn', 'insan', 'violent', 'mob', 'crazi', 'chant', 'singer', 'unfortun', 'stay', 'absurd', 'whole', 'time', 'gener', 'narr', 'eventu', 'make', 'put', 'even', 'era', 'turn', 'cryptic', 'dialogu', 'would', 'make', 'shakespear', 'seem', 'easi', 'third', 'grader', 'technic', 'level', 'better', 'might', 'think', 'good', 'cinematographi', 'futur', 'great', 'vilmo', 'zsigmond', 'futur', 'star', 'salli', 'kirkland', 'freder', 'forrest', 'seen', 'briefli']


# Embedding

In [21]:
vocab = set()
reviews = [tokenise(text) for text in data['review']]

In [22]:
for txt in reviews:
    for word in txt:
        vocab.add(word)

In [25]:
word2idx = {j for i,j in enumerate(vocab)}
idx2word = np.array(vocab)

In [30]:
def vectorize(text):
    temp = np.zeros(len(vocab))
    for word in text:
        temp[word2idx[word]]+=1
    return temp

In [42]:
freq_matrix = np.zeros((len(data['review']), len(vocab)), dtype = 'b1')

for i in range(len(data['review'])):
    for word in reviews[i]:
        freq[i][word2idx[word]]+=1

MemoryError: Unable to allocate 2.15 GiB for an array with shape (25000, 92247) and data type bool

{'', 'jabbathehutesquir', 'jawsand', 'crackerbut', 'pea', '4000', 'thereaft', 'maïga', 'michellarr', 'documentarytyp', 'supranatur', 'gunilla', 'wristwatch', 'numberless', 'tarintino', 'femal', 'ruinedor', 'feebr', 'categoris', 'nonfrenchspeak', 'beehiv', 'amishesquir', 'headeyi', 'thuddingli', 'zukov', 'createbr', 'leaveit', 'farrakhan', 'mcandrew', 'amoebalik', 'marian', 'glacial', 'pinnocioesqu', '13thversion', 'richlyproduc', 'ooe', 'requirementtoo', 'etcmarcel', 'fartoocarelessli', 'leza', 'supermanh', 'stiffleg', 'serieswhen', 'sabubr', 'topmelodramat', 'juliann', 'insidi', 'stationthey', 'shadowbox', 'piven', 'caulder', 'againi', 'lilienth', 'acapulco', 'thingalong', 'menfolk', 'fanslugosi', 'soundera', 'lorraina', 'mummylik', 'axl', 'chivalr', 'birtwhistl', '62006', 'wardad', 'ammanda', 'lovebird', 'musicdanc', 'felton', 'interestingi', '93minut', 'endwhich', 'neither', 'sirin', 'threatbr', 'jacobson', 'prewwii', 'golding', 'cini', 'onehorn', 'achcha', 'unknownit', 'eventempere

0        ['stori', 'man', 'unnatur', 'feel', 'pig', 'st...
1        ['airport', '77', 'start', 'brand', 'new', 'lu...
2        ['film', 'lack', 'someth', 'couldnt', 'put', '...
3        ['sorri', 'everyon', 'know', 'suppos', 'art', ...
4        ['littl', 'parent', 'took', 'along', 'theater'...
                               ...                        
24995    ['see', 'vote', 'averag', 'pretti', 'low', 'fa...
24996    ['plot', 'wretch', 'unbeliev', 'twist', 'howev...
24997    ['amaz', 'movieand', 'other', 'averag', '5', '...
24998    ['christma', 'togeth', 'actual', 'came', 'time...
24999    ['workingclass', 'romant', 'drama', 'director'...
Name: review, Length: 25000, dtype: object
